# Main analysis on api collected data to follow

In [167]:
# import main libraries

import os
import requests
import pandas as pd
import json
from pathlib import Path
import numpy as np
from datetime import datetime
import dateutil.parser
import urllib.request
from dotenv import load_dotenv

# import data viz libraries

import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, CustomJS, MultiSelect, CheckboxGroup
from bokeh.models.widgets import Tabs, Panel

%matplotlib inline

# import interactive panels libraries

import panel as pn
from panel.interact import interact
from panel import widgets
# pn.extension('plotly')
pn.extension()



In [22]:
#import hvplot.pandas

In [23]:
# Read the API keys
load_dotenv()

True

In [24]:
##########################################################
# Create data structures used throughout
##########################################################
# Define positions (indexes, algorithmitic portfolios, assets to evaluate
# Define currencies to review performance against
# Define evaluation calculations to perform
# All are manually coded in for first MVP

asset1 = 'SP500'
asset2 = 'SP500BDT'
asset5 = 'GlobalArtIndex'
asset6 = 'XAUUSD'
asset7 = 'BTCUSD'

currency1 = 'USD'
currency2 = 'XAU'
currency3 = 'BTC'

In [25]:
start_date = '2010-01-01'
symbols = "SP500BDT.INDX"
asset_type = "S&P500 Bond Index"

#symbols = "GSPC.INDX"
#asset_type = "S&P500 Index"

symbols = "BCOMGCTR.INDX"
asset_type = "Gold Total Return Index"

symbols = "BTC-USD.CC"
asset_type = "Bitcoin EOD Price"

symbols = "GC.COMM"
asset_type = "Gold EOD Price"

eod_api_key = os.getenv("EOD_API_KEY")
eod_option = {'api_token': eod_api_key, 'order': 'd', 'fmt': 'json', 'from': start_date}

eod_request_url = "https://eodhistoricaldata.com/api/eod/" + symbols
#eod_request_url = "https://eodhistoricaldata.com/api/eod/MSFT.US?api_token=5fa212b3754234.49376057"
eod_request_url

# Execute get request
eod_response_data = requests.get(eod_request_url, eod_option)
#response_content = eod_response_data.content
#print(response_content)


In [195]:
####################################################################
# Main Market Dataset Collection
# Equities
# Precious Metals
# Crpyto currencies
####################################################################



to_eod_symbol_dict = {
                    'SP500BDT' : 'SP500BDT.INDX',
                    'SP500' : 'GSPC.INDX',
                    'BTCUSD' : 'BTC-USD.CC',
                    'XAUUSD': 'GC.COMM'}
#from_eod_symbol_dict = {}
#for key, value in to_eod_symbol_dict.items():
#    from_eod_symbol_dict[value] = key

#print(to_eod_symbol_dict[asset1])

start_date = '2010-01-01'


def get_data(symbols, start_date):
 #   symbols_dict = {'SP500BDT.INDX':'S&P500 Bond Index', 
 #                   'GSPC.INDX':'S&P500 Index', 
 #                   'BCOMGCTR.INDX':'Gold Total Return Index', 
 #                   'BTC-USD.CC':'Bitcoin EOD Price', 
 #                   'GC.COMM':'Gold EOD Price',}
    
    eod_api_key = os.getenv("EOD_API_KEY")
    eod_option = {'api_token': eod_api_key, 'order': 'd', 'fmt': 'json', 'from': start_date}

    eod_request_url = "https://eodhistoricaldata.com/api/eod/" + symbols

    # Execute get request
    eod_response_data = requests.get(eod_request_url, eod_option)
    eod_data = eod_response_data.json()
    #print(json.dumps(data, indent=4))

    column_list = ['Date', 'Asset', 'AdjClosePrice']
    eod_df = pd.DataFrame(columns=column_list)

    for dict in eod_data: 
        for list in dict: 
            if list == "date":
                temp_date = datetime.strptime(dict[list], '%Y-%m-%d')
            elif list == "adjusted_close":
                temp_close = dict[list] 
            
        new_row = {'Date':temp_date, 'Asset':symbols, 'AdjClosePrice':temp_close}
        eod_df = eod_df.append(new_row, ignore_index=True)                 
    
    eod_df = eod_df.set_index(['Date']) 
    eod_df = eod_df.sort_index()
    return eod_df


asset1_data_USD_df = get_data(to_eod_symbol_dict[asset1],start_date)
print(asset1)
asset1_data_USD_df['Asset'] = asset1
#asset1_data_USD_df['Asset'] = asset1_data_USD_df['Asset'].replace([to_eod_symbol_dict[asset1],'SP500'])
#asset1_data_USD_df['Asset'].replace([to_eod_symbol_dict[asset1],asset1],inplace=True)




#asset1_data_USD_df.head()

SP500


In [196]:
asset2_data_USD_df = get_data(to_eod_symbol_dict[asset2],start_date)
print(asset2)
asset2_data_USD_df['Asset'] = asset2
#asset2_data_USD_df.head()

SP500BDT


In [197]:
asset6_data_USD_df = get_data(to_eod_symbol_dict[asset6],start_date)
print(asset6)
asset6_data_USD_df['Asset'] = asset6
#asset6_data_USD_df.head()

XAUUSD


In [198]:
asset7_data_USD_df = get_data(to_eod_symbol_dict[asset7],start_date)
print(asset7)
asset7_data_USD_df['Asset'] = asset7
#asset7_data_USD_df.head()

BTCUSD


In [199]:
## Fetch ART Data ##
art_file_path = Path("../data_imports/Artprice_Indexes_Quarterly_data_Base100_January1998.csv")
art_index = pd.read_csv(art_file_path, encoding='utf-8', parse_dates=True, infer_datetime_format=True)
art_index.rename(columns=({"Unnamed: 0" : "Date"}), inplace=True)
art_index.dropna()
art_index.set_index("Date", drop=True, inplace=True)
#art_index.tail()
# first select art index of choice and scale up price by 100 & add position name column
gblart_index_col_list = ['Asset','AdjClosePrice']
gblart_index_df = pd.DataFrame(columns=gblart_index_col_list)
gblart_index_df['AdjClosePrice'] = (art_index.loc[:,'Global Index (USD)'])*100
gblart_index_df['Asset'] = 'GlobalArtIndex'
asset5_data_USD_df = gblart_index_df
#asset5_data_USD_df.head()


In [200]:
####################################################################
# EVERYTHING ABOVE WILL BE REVISED ONCE DATASETS FUNCTIONS CREATED
# BELOW STARTS THE FINAL VERSION OF ANALYTICS AND VISUALIZATIONS
####################################################################

# Define calculation, currency and asset structure for 
# currency conversion and main analysis

# hard code for now to test

calc_list = ['STD_200D','MA_200D','DailyReturns','CumulReturns']

asset1_calc_USD_df = pd.DataFrame()
asset1_calc_BTC_df = pd.DataFrame()
asset1_calc_XAU_df = pd.DataFrame()

asset2_calc_USD_df = pd.DataFrame()
asset2_calc_BTC_df = pd.DataFrame()
asset2_calc_XAU_df = pd.DataFrame()

asset5_calc_USD_df = pd.DataFrame()
asset5_calc_BTC_df = pd.DataFrame()
asset5_calc_XAU_df = pd.DataFrame()

asset6_calc_USD_df = pd.DataFrame()
asset6_calc_BTC_df = pd.DataFrame()
asset6_calc_XAU_df = pd.DataFrame()

asset7_calc_USD_df = pd.DataFrame()
asset7_calc_BTC_df = pd.DataFrame()
asset7_calc_XAU_df = pd.DataFrame()

eval_asset_dict = {
                asset1 : {
                        'data' : asset1_data_USD_df,
                        'USD' : asset1_calc_USD_df,
                        'BTC' : asset1_calc_BTC_df,
                        'XAU' : asset1_calc_XAU_df
                        }}


In [201]:
####################################################################
# CURRENCY CONVERSION FUNCTION WILL BE HERE IN THIS CELL
####################################################################




In [202]:
####################################################################
# ASSET ANALYSIS FUNCTION 
####################################################################

def analyze_asset(calc_list,wip_calc_df):
    """
    Market price data analysis and asset return calcs
    as defined in calc_list input on provided wip_calc_df
    of format: date, asset, adjclose
    """
    for calc_type in calc_list:
        # Do analysis on price trends before return calcs 
        # to align return calcs with same window
        if calc_type == 'STD_200D':
            wip_calc_df[calc_type] = wip_calc_df['AdjClosePrice'].rolling(window=200).std()
        elif calc_type == 'MA_200D':
            wip_calc_df[calc_type] = wip_calc_df['AdjClosePrice'].rolling(window=200).mean()
            wip_calc_df.dropna(inplace=True)
        elif calc_type == 'DailyReturns':
            wip_calc_df[calc_type] = wip_calc_df['AdjClosePrice'].pct_change()
            wip_calc_df.dropna(inplace=True)
        elif calc_type == 'CumulReturns':
            wip_calc_df[calc_type] = (1+wip_calc_df['DailyReturns']).cumprod() - 1
        else:
            print(f"What is {calc_type} doing here?!")        
    return(wip_calc_df)


In [230]:
####################################################################
# MAIN ANALYSIS LOOP 
####################################################################
# hard code for now
wip_calc_df = asset1_data_USD_df
analyze_asset(calc_list,wip_calc_df)
asset1_calc_USD_df = wip_calc_df
asset1_calc_BTC_df = wip_calc_df
asset1_calc_XAU_df = wip_calc_df

wip_calc_df = asset2_data_USD_df
analyze_asset(calc_list,wip_calc_df)
asset2_calc_USD_df = wip_calc_df
asset2_calc_BTC_df = wip_calc_df
asset2_calc_XAU_df = wip_calc_df

wip_calc_df = asset5_data_USD_df
analyze_asset(calc_list,wip_calc_df)
asset5_calc_USD_df = wip_calc_df
asset5_calc_BTC_df = wip_calc_df
asset5_calc_XAU_df = wip_calc_df

wip_calc_df = asset6_data_USD_df
analyze_asset(calc_list,wip_calc_df)
asset6_calc_USD_df = wip_calc_df
asset6_calc_BTC_df = wip_calc_df
asset6_calc_XAU_df = wip_calc_df

wip_calc_df = asset7_data_USD_df
analyze_asset(calc_list,wip_calc_df)
asset7_calc_USD_df = wip_calc_df
asset7_calc_BTC_df = wip_calc_df
asset7_calc_XAU_df = wip_calc_df

"""
#loop for each asset: access proper df, currency convert, and analyze
for asset in eval_asset_list:
    print(f"Analyzing {asset}")
    for currency in currency_list: 
        # Run conversion on USD market price datasets to normalize price to alt currency
        if currency == 'USD':
            print(f"Market data in USD, not converting")
            wip_calc_df = asset1_data_USD_df
            analyze_asset(calc_list,wip_calc_df)
            asset1_calc_USD_df = wip_calc_df
        elif currency == 'XAU':
            print(f"Converting market price data to {currency}")
            wip_calc_df = asset1_data_USD_df
            analyze_asset(calc_list,wip_calc_df)
            asset1_calc_XAU_df = wip_calc_df
        elif currency == 'BTC':
            print(f"Converting market price data to {currency}")
            wip_calc_df = asset1_data_USD_df
            analyze_asset(calc_list,wip_calc_df)
            asset1_calc_BTC_df = wip_calc_df
        else:
            print(f"What is {currency} doing here?!")
"""
print(asset1_calc_USD_df.head())
#print(asset2_calc_XAU_df.head())
#print(asset5_calc_BTC_df.head())

            Asset  AdjClosePrice   STD_200D      MA_200D  DailyReturns  \
Date                                                                     
2011-08-04  SP500      1200.0699  50.084673  1286.346695     -0.047821   
2011-08-05  SP500      1199.3800  49.876489  1286.452745     -0.000575   
2011-08-08  SP500      1119.4600  50.705395  1286.148745     -0.066634   
2011-08-09  SP500      1172.5300  50.818520  1286.095995      0.047407   
2011-08-10  SP500      1120.7600  51.662871  1285.771695     -0.044152   

            CumulReturns  
Date                      
2011-08-04     -0.047821  
2011-08-05     -0.048368  
2011-08-08     -0.111779  
2011-08-09     -0.069672  
2011-08-10     -0.110748  


In [204]:
####################################################################
# CELL RESERVED FOR FINAL DF MASSAGE AS NEEDED FOR VISUALIZATIONS 
####################################################################

# After looping can concat into long frame as needed by visualizations
# lcombo_USD_df = pd.concat(frames)

In [205]:
####################################################################
# HENRY SCRATCH PAD SAVE FOR NEAR TERM REFERENCE
####################################################################
# test merge trying zero fill NaN locations to help with line plot below -- spoiler alert -- doesnt work
#test_combo_df = sp500_daily_returns.merge(gblart_returns_df,how='left',left_index=True,right_index=True).fillna(0)
# long head here to verify that its joining where there's valid art data
#test_combo_df.head(100)

#def sanitize_dataset(dataset_df):
#    """Insure data sorted ascending"""
#    dataset_df = dataset_df.sort_index()
#    print(dataset_df.head())
#    return(dataset_df)
#
## Sweep each position,for each currency, for each calculation
#for pos in eval_pos_list:
#    # first do finalize sanitation step on incoming position dataset
#    print(type(pos))
#    sanitize_dataset(pos)
#    for currency in currency_df_list:
#        if (currency == USD):


In [206]:
####################################################################
# THIS CELL IS START OF VISUALIZATION FUNCTIONS 
#
####################################################################
# Define Panel Visualization Functions
# Number of plot functions right now = 5 calcs/position * 2timeframes * 6 positions = 60!!!!

#def 
#art_index_std_bar = px.bar(art_index_std, title="Art Price STD 1998-2020")
#art_index_std_bar

In [243]:
# Define Multi Select Interaction

def multi_select_assets():
    options = [("1", "S&P 500"), ("2", "S&P500 Bond Index"), ("3", "Coming Soon"), ("4", "Coming Soon"), ("5", "Global Art Index"), ("6", "Gold"), ("7", "BTC USD")]
    
    multi_select = MultiSelect(value=["1", "2"], options=options)
    multi_select.js_on_change("value", 
                              CustomJS(
        code="""
        console.log('multi_select: value=' + this.value, this.toString())
        """)
                             )
    return multi_select
   
def multi_select_currency():
    labels = ["USD", "Gold (XAU)", "BTC"]
    
    checkbox_group = CheckboxGroup(labels=labels, active=[0])
    checkbox_group.js_on_click(CustomJS(code="""
    console.log('checkbox_group: active=' + this.active, this.toString())
    """))

    return checkbox_group
    

# Define Plot Functions 

def price_performance(df):
    
    # Output the visualization directly in the notebook
    output_notebook()
    
        
    # Parse Column data
    source = ColumnDataSource(data=df)
    
    # Create the figure object
    pp_fig = figure(
        x_axis_type='datetime',
        title=("Price Performance"),
        plot_height=400, plot_width=900,
        x_axis_label='Date', y_axis_label='Returns',
    )
 
    # Draw the figure
  
    pp_fig.line(
        x='Date',
        y="AdjClosePrice",
        source=source,
        color='green', 
        width=0.75, 
        legend='Price'
    )
    
    pp_fig.line(
        x='Date',
        y="CumulReturns",
        source=source,
        color='purple', 
        width=0.75, 
        legend='Cumulative Returns'
    )
    
    pp_fig.line(
        x='Date',
        y="STD_200D",
        source=source,
        color='red', 
        width=0.75, 
        legend='200 Day STD'
    )
        
    pp_fig.line(
        x='Date',
        y="MA_200D",
        source=source,
        color='blue', 
        width=0.75, 
        legend='200 Day Moving Average'
    )
    

        
    pp_fig.legend.location = 'top_left'
    
    pp_pane = pn.pane.Bokeh(pp_fig)
    
    return pp_pane

def cum_returns(df):
    
    # Output the visualization directly in the notebook
    output_notebook()
    
        
    # Parse Column data
    source = ColumnDataSource(data=df)
    
    # Create the figure object
    cr_fig = figure(
        x_axis_type='datetime',
        title=("Cumulative Returns"),
        plot_height=400, plot_width=800,
        x_axis_label='Date', y_axis_label='Returns',
    )
 
    # Draw the figure
  
    cr_fig.line(
        x='Date',
        y="CumulReturns",
        source=source,
        color='red', 
        width=0.75, 
        legend='Price'
    )
    
    cr_fig.legend.location = 'top_left'
    
    cr_pane = pn.pane.Bokeh(cr_fig)
    
    return cr_pane
    

# Creating the Price Performance Panes

sp500_pp_pane = price_performance(asset1_calc_USD_df)
sp500_bond_pp_pane = price_performance(asset2_calc_USD_df)
# sp500_pane = price_performance(asset3_calc_USD_df)
# sp500_pane = price_performance(asset4_calc_USD_df)
art_pp_pane = price_performance(asset5_calc_USD_df)
gold_pp_pane = price_performance(asset6_calc_USD_df)
btc_pp_pane = price_performance(asset7_calc_USD_df)

# Creating the Cumulative Returns Panes

# sp500_cr_pane = cum_returns(asset1_calc_USD_df)
# sp500_bond_cr_pane = cum_returns(asset2_calc_USD_df)
# sp500_pane = cum_returns(asset3_calc_USD_df)
# sp500_pane = cum_returns(asset4_calc_USD_df)
# art_cr_pane = cum_returns(asset5_calc_USD_df)
# gold_cr_pane = cum_returns(asset6_calc_USD_df)
# btc_cr_pane = cum_returns(asset7_calc_USD_df)


    

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

In [244]:

# Create Panel Dashboard

welcome_column = pn.Column(
    "# On The Money Portfolio Management Tool",
    "### We aim to maximize retail investors’ alpha over the long term by helping them make more informed decisions.",
    
    
    "#### Welcome to the asset analysis dashboard. Use the tabs above to navigate through the different sections."
    "Use the input below to see how your ideas and assets compare with the benchmarks.",
    
    "### Select the Asset Classes you wish to view:",
    "##### *(Press and Hold Command to select multiple)*",
    multi_select_assets(),
    multi_select_currency()
)

price_performance = pn.Column(
    "### S&P 500",
    sp500_pp_pane,
    "### S&P 500 Bond Index",
    sp500_bond_pp_pane,
    "### Global Art Index",
    art_pp_pane,
    "### Gold",
    gold_pp_pane,
    "### Bitcoin",
    btc_pp_pane
)

cum_return = pn.Column(
    "### S&P 500",
    sp500_rc_pane,
    "### S&P 500 Bond Index",
    sp500_bond_cr_pane,
    "### Global Art Index",
    art_cr_pane,
    "### Gold",
    gold_cr_pane,
    "### Bitcoin",
    btc_cr_pane
)

portfolio_dashboard = pn.Tabs(
    (
        "Welcome",
        welcome_column
    ),
    (
        "Price Performance Of Asset Classes",
        price_performance
    ),
    (
        "Cumulative Return Of Asset Classes",
        cum_return
    )
)

portfolio_dashboard

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Markdown(str)
        [3] Markdown(str)
        [4] Markdown(str)
        [5] Bokeh(MultiSelect)
        [6] Bokeh(CheckboxGroup)
    [1] Column
        [0] Markdown(str)
        [1] Bokeh(Figure)
        [2] Markdown(str)
        [3] Bokeh(Figure)
        [4] Markdown(str)
        [5] Bokeh(Figure)
        [6] Markdown(str)
        [7] Bokeh(Figure)
        [8] Markdown(str)
        [9] Bokeh(Figure)
    [2] Column
        [0] Markdown(str)
        [1] Bokeh(Figure)
        [2] Markdown(str)
        [3] Bokeh(Figure)
        [4] Markdown(str)
        [5] Bokeh(Figure)
        [6] Markdown(str)
        [7] Bokeh(Figure)
        [8] Markdown(str)
        [9] Bokeh(Figure)

In [ ]:
portfolio_dashboard.servable()

In [214]:
portfolio_dashboard.app('localhost:8888')